In [12]:
import numpy as np

def fcn(p):
    x, y, z = p
    return (x-1)**2 + (y+2)**2 + (z-3)**2 + 0.5*(np.sin(x*y) + np.sin(y*z))

def nm(fn, smp, a=1.0, b=1.0, g=0.5, iters=1000, tol=1e-5):
    s = np.array(smp, dtype=float)
    n = len(s) - 1

    for _ in range(iters):
        vals = np.array([fn(p) for p in s])
        idx = np.argsort(vals)
        s = s[idx]
        vals = vals[idx]

        if np.std(vals) < tol:
            break

        c = np.mean(s[:-1], axis=0)
        xr = c + a*(c - s[-1])
        fr = fn(xr)

        if fr < vals[0]:
            xe = c + b*(c - xr)
            s[-1] = xe if fn(xe) < fr else xr
        elif fr < vals[-2]:
            s[-1] = xr
        else:
            if fr < vals[-1]:
                xc = c + g*(xr - c)
                s[-1] = xc if fn(xc) <= fr else shrink(s, g)
            else:
                xcc = c - g*(c - s[-1])
                s[-1] = xcc if fn(xcc) < vals[-1] else shrink(s, g)

    return s[np.argmin([fn(p) for p in s])]

def shrink(s, g):
    xb = s[0]
    return np.array([xb + g*(p - xb) for p in s])

smp = [
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [0,0,1]
]

res = nm(fcn, smp)
print("Min:", res)
print("Val:", fcn(res))

Min: [ 0.82562213 -2.28528643  3.24284904]
Val: -0.7561777543005382


In [2]:
import numpy as np

# 定义目标函数：Shubert 类似函数
def objective(pos):
    x, y = pos
    return -x * np.sin(np.sqrt(abs(x))) - y * np.sin(np.sqrt(abs(y)))

def simulated_annealing_optimize():
    # 初始参数设定
    initial_temp = 1000.0
    final_temp = 1e-6
    max_iterations = 10000
    cooling_rate = 1000.0
    move_radius = 50.0
    
    # 初始化搜索起点
    position = np.random.uniform(-500, 500, size=2)
    best_value = objective(position)

    temperature = initial_temp
    iteration = 0

    while temperature > final_temp and iteration < max_iterations:
        # 扰动产生新候选点
        candidate = position + np.random.uniform(-move_radius, move_radius, size=2)
        candidate = np.clip(candidate, -500, 500)
        candidate_value = objective(candidate)
        
        diff = candidate_value - best_value
        
        # Metropolis准则决定是否接受
        if diff < 0 or np.random.rand() < np.exp(-diff / temperature):
            position = candidate
            best_value = candidate_value
        
        # 温度下降（指数冷却）
        temperature = initial_temp * np.exp(-iteration / cooling_rate)
        iteration += 1

    print(f"Start Temp: {initial_temp}")
    print(f"End Temp: {temperature}")
    return position, best_value

# 启动优化过程
opt_pos, opt_val = simulated_annealing_optimize()
print(f"Optimal Position: ({opt_pos[0]:.2f}, {opt_pos[1]:.2f})")
print(f"Objective Value: {opt_val:.2f}")

Start Temp: 1000.0
End Temp: 0.04544535239978074
Optimal Position: (420.71, 420.48)
Objective Value: -837.93
